In [6]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
def write_to_file(l, number):
    with open(f"/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/split_cube/part-{number}", 'w') as f:
        for item in l:
            f.write(item+'\n')

### Find files that must be cubed

In [2]:
db = pd.read_pickle("/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/calculations_databse")


In [ ]:
workdirs = db.loc[db['status'] == 'converged', 'workdir']

In [ ]:
dde = []
for w in workdirs:
    dens_file = os.path.join(w, 'DENSITY')
    if not os.path.isfile(dens_file):
        dde.append(w)

In [ ]:
cube = []
to_cube = []
for w in workdirs:
    cube_file = os.path.join(w, 'DENSITY.cube')
    if os.path.isfile(cube_file):
        cube.append(w)
    else:
        to_cube.append(w)

### divide in small packages that fit in 30 min qos

In [ ]:
start = 6
for i in range(26):
    part = to_cube[start:start+20]
    write_to_file(part, i)
    start += 20

In [ ]:
len(to_cube[6:])/20

In [ ]:
write_to_file(to_cube[519+7:], 26)

### Move to cube-files directory

In [10]:
fde = []
for comp in list(set(db['comp_name'])):
    # make cube-files directory
    cube_path = os.path.join("/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38", f"{comp}/cube-files")
    
    os.makedirs(cube_path, exist_ok=True)
    
    # move cube-files
    comp_wds = db.loc[db['comp_name'] == comp, 'workdir'] # list of all directories for each compound
    lam_vals = db.loc[db['comp_name'] == comp, 'lam_val']
    for l, wd in zip(lam_vals, comp_wds):
        source = os.path.join(wd, "DENSITY.cube")
        dest = os.path.join(cube_path, f"{l}.cube")
        if os.path.isfile(source):
            shutil.move(source, dest)
        else:
            fde.append(wd)
        #print(source)
        #print(dest)

### Make list of cube-dirs to tar

In [15]:
cube_dirs = []
for comp in list(set(db['comp_name'])):
    # make cube-files directory
    cube_path = os.path.join(".", f"{comp}/cube-files")
    cube_dirs.append(cube_path)

In [17]:
with open(f"/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/cubes2tar", 'w') as f:
    for item in cube_dirs:
        f.write(item+'\n')